In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from scipy import stats
from scipy.stats import contingency
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/kaggle/input/Glassdoor Gender Pay Gap.csv')

In [ ]:
data

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=data, x='JobTitle', hue='Gender')

In [ ]:
jobData = data.groupby(['JobTitle', 'Gender']).size().unstack(fill_value=0)
#ourData['Total']= ourData['Female']+ourData['Male']
jobData

In [ ]:
sns.heatmap(jobData,cmap='RdBu')

from the plots it is shown that there are dependancies between gender and some jobs especially sofware engineer, manager and marketing associate positions.

In [ ]:
jobDataProp =jobData/jobData.sum()
jobDataProp

In [ ]:
x2, p_value, dof, expected_freq = stats.chi2_contingency(jobData)
print(f"X2: {x2}, P value: {p_value}, degree of freedom: {dof}")

Because P value is < 0.05 we rejet the null hypothesis which is the job is prefered equaly by males and females. Furthuer tests are needed to determine which jobs are dependant on gender.


In [ ]:
def two_sample_proportion_test(prop1, prop2, sample_size1, sample_size2, significance_level):
    # prop1 and prop2 are the  frequencies of both populations 
    # sample_size1 and sample_size2 are the total number of poplulation 1 and population 2 
    # singificane_level is the critical value/number_of_tests
    
    mu_0 = 0 # null hypothesis mean p1-p2 = 0
    standard_error = np.sqrt((prop1*(1-prop1)/sample_size1) + (prop2*(1-prop2)/sample_size2))
    p_value = stats.norm.sf(abs(prop1 - prop2), loc=mu_0, scale=standard_error) # P_value of the null hypothyses 
    
    
    # Calculating probability of errors
    alternative_mu = prop1 - prop2
    if alternative_mu > 0:
        critical_value = stats.norm.isf(significance_level, loc=mu_0, scale=standard_error) # if alternative is prop1-prop2> 0 then the critival value is 1-significance level it can be considered inverse of the null test 
    else:
        critical_value = stats.norm.ppf(significance_level, loc=mu_0, scale=standard_error)# if alternative is prop1-prop2< 0 then the critival value is significance level
    
    
    if critical_value > alternative_mu:
        beta_error = stats.norm.sf(critical_value, loc=alternative_mu, scale=standard_error)
    else:
        beta_error = stats.norm.cdf(critical_value, loc=alternative_mu, scale=standard_error)
        
    confusion_matrix = np.array([[1-significance_level, significance_level],[beta_error, 1-beta_error]])
    sns.heatmap(confusion_matrix, annot=True, xticklabels=['Fail to reject', 'Reject'], yticklabels=['Ho True', 'Ho False'])
    plt.show()
    
    return p_value, beta_error

In [ ]:
#using reduced significance level

significance_level = 0.05/len(jobData)
for job in jobData.index:
    print(f"\n{job}")
    p1, p2 = jobDataProp.loc[job]
    n1, n2 = jobData.sum()
    if (n1*p1 >= 10) and (n2*p2 >= 10): 
        p_value, beta = two_sample_proportion_test(p1, p2, n1, n2, significance_level)
        print(f"P Value = {p_value}, beta = {beta}")
        message = "Reject" if p_value < significance_level else "Fail to reject"
        print(message)

    else:
        print("no results")

After studing each job it is shown that all the jobs are not dependant on Gender except for **manager** and **marketing associate** positions. Also, the test for software engineering job position could not be made as it did not satisfy the test criteria which is n*p >10.



# 2 - Department

In [ ]:
departmentData = data.groupby(['Dept', 'Gender']).size().unstack(fill_value=0)
#ourData['Total']= ourData['Female']+ourData['Male']
departmentData

In [ ]:
departmentDataProp = departmentData/departmentData.sum()
departmentDataProp

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=data, x='Dept', hue='Gender')

In [ ]:
sns.heatmap(departmentData,cmap='RdBu')

from the plots we can have an initial guess that there is no dependancy between departments and gender.

In [ ]:
x2, p_value, dof, expected_freq = stats.chi2_contingency(departmentData)
print(f"X2: {x2}, P value: {p_value}, degree of freedom: {dof}")


Since p value is > than 0.05 then we fail to reject which means that there is no prefrence between males and females when it comes to gender.

In [ ]:
significance_level = 0.05/len(departmentData)
for dep in departmentData.index:
    print(f"\n{dep:=^80}")
    p1, p2 = departmentDataProp.loc[dep]
    n1, n2 = departmentData.sum()
    if (n1*p1 >= 10) and (n2*p2 >= 10): 
        p_value, beta = two_sample_proportion_test(p1, p2, n1, n2, significance_level)
        print(f"P Value = {p_value}, beta = {beta}")
        message = "Reject" if p_value < significance_level else "Fail to reject"
        print(message)
    else:
        print("no results")

form studing each deprtment it is shown that there is no dependancy between the given departments and gender as we failed to reject the null hypothesis for all the departments.